In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
labels_files = pd.read_csv('/Users/gyony/Downloads/archive/animal_data_img.csv')
labels_files

In [ ]:
len(labels_files.Label.value_counts())

In [ ]:
len(labels_files.Animal_Type.value_counts())

In [ ]:
labels_files.drop("Label", axis = 1, inplace = True)
labels_files

In [ ]:
filenames = os.listdir("/Users/gyony/Downloads/archive/animal_images")

In [ ]:
labels_files['Animal_Type'].value_counts().plot.bar()

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(labels_files, test_size=0.20, random_state=2021)
train_df, validate_df = train_test_split(train_df, test_size=0.20, random_state=2022)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [ ]:
IMAGE_SIZE=(224, 224)
IMAGE_CHANNELS=3
batch_size = 128

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "/Users/gyony/Downloads/archive/animal_images/", 
    x_col='Image_File',
    y_col='Animal_Type',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "/Users/gyony/Downloads/archive/animal_images/", 
    x_col='Image_File',
    y_col='Animal_Type',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

MODEL 1

In [ ]:
MODEL_SAVE_FOLDER_PATH = './model/'

if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'animal-' + '{epoch:02d}-{val_loss:.4f}.cc'

checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
earlystop = EarlyStopping(monitor = 'val_accuracy', patience = 3)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(7, activation='softmax')) 


#model.add(Dense(512, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
#model.add(Dense(7, activation='softmax')) 


input_shape = [None, 224, 224, 3] 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape)
model.summary()

In [ ]:
  history1 = model.fit(
      x = train_generator,
      epochs = 30,
      validation_data = validation_generator,
      validation_freq = 1,
      callbacks = [earlystop, learning_rate_reduction, checkpoint]
  )

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history1.history1['loss'], color='b', label="Training loss")
ax1.plot(history1.history1['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, 25, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history1.history1['accuracy'], color='b', label="Training accuracy")
ax2.plot(history1.history1['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, 25, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

In [ ]:
predict = model.predict(test_generator)

In [ ]:
np.argmax(predict, axis=-1)

In [ ]:
test_df['category'] = np.argmax(predict, axis=-1)

In [ ]:
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

In [ ]:
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['Image_File']
    Animal_Type = row['Animal_Type']
    category = row['category']
    img = load_img("/Users/gyony/Downloads/archive/animal_images/"+filename, target_size=IMAGE_SIZE)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel( 'Predicted:'+'(' +"{}".format(category) + ')' + ' True:'+'(' +"{}".format(Animal_Type) + ')')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_df['Animal_Type'], test_df['category'], target_names=types))

In [ ]:
 from sklearn.metrics import accuracy_score
accuracy_score(test_df['Animal_Type'], test_df['category'])

MODEL 2

In [ ]:
MODEL_SAVE_FOLDER_PATH = './model/'

if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'animal-' + '{epoch:02d}-{val_loss:.4f}.cc'

checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
earlystop = EarlyStopping(monitor = 'val_accuracy', patience = 3)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax')) 


input_shape = [None, 224, 224, 3] 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.build(input_shape)
model.summary()

In [ ]:
  history = model.fit(
      x = train_generator,
      epochs = 30,
      validation_data = validation_generator,
      validation_freq = 1,
      callbacks = [earlystop, learning_rate_reduction, checkpoint]
  )

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, 25, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, 25, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

In [ ]:
predict = model.predict(test_generator)

In [ ]:
np.argmax(predict, axis=-1)

In [ ]:
test_df['category'] = np.argmax(predict, axis=-1)

In [ ]:
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

In [ ]:
test_df

In [ ]:
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['Image_File']
    Animal_Type = row['Animal_Type']
    category = row['category']
    img = load_img("/Users/gyony/Downloads/archive/animal_images/"+filename, target_size=IMAGE_SIZE)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel( 'Predicted:'+'(' +"{}".format(category) + ')' + ' True:'+'(' +"{}".format(Animal_Type) + ')')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_df['Animal_Type'], test_df['category'], target_names=types))

In [ ]:
 from sklearn.metrics import accuracy_score
accuracy_score(test_df['Animal_Type'], test_df['category'])

MODEL 3 

In [ ]:
MODEL_SAVE_FOLDER_PATH = './model/'

if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)

model_path = MODEL_SAVE_FOLDER_PATH + 'animal-' + '{epoch:02d}-{val_loss:.4f}.hdf5'

checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1, save_best_only=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
earlystop = EarlyStopping(monitor = 'val_accuracy', patience = 3)

In [ ]:
model_url = 'https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/5'
input_shape = [None, 224, 224, 3] 

# setup the output shape 
output_shape = 7

# setup the layers
model = Sequential([
  hub.KerasLayer(model_url),
  Dense(units = output_shape, activation = 'softmax') 
])

# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


# Build the model
model.build(input_shape)

In [ ]:
  history = model.fit(
      x = train_generator,
      epochs = 30,
      validation_data = validation_generator,
      validation_freq = 1,
      callbacks = [earlystop, learning_rate_reduction, checkpoint]
  )

In [ ]:
model.save_weights("model.hdf5")

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, 7, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, 7, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

Test

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "/Users/gyony/Downloads/archive/animal_images/", 
    x_col='Image_File',
    y_col='Animal_Type',
    class_mode='categorical',
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)

In [ ]:
predict = model.predict(test_generator)

In [ ]:
np.argmax(predict, axis=-1)

In [ ]:
test_df['category'] = np.argmax(predict, axis=-1)

In [ ]:
label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

In [ ]:
test_df

In [ ]:
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['Image_File']
    Animal_Type = row['Animal_Type']
    category = row['category']
    img = load_img("/Users/gyony/Downloads/archive/animal_images/"+filename, target_size=IMAGE_SIZE)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel( 'Predicted:'+'(' +"{}".format(category) + ')' + ' True:'+'(' +"{}".format(Animal_Type) + ')')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_df['Animal_Type'], test_df['category'], target_names=types))

In [ ]:
 from sklearn.metrics import accuracy_score
accuracy_score(test_df['Animal_Type'], test_df['category'])